In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [ ]:
DATADIR = r'/content/data256/train'
CATEGORIES = [0, 1]
SIZE = 256

labels = pd.read_csv(r'/content/drive/MyDrive/train.csv')

X = []
y = []
files_array = []
def creating_training_dataset(X,y,DATADIR, labels, SIZE):
    path = os.path.join(DATADIR)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        img_array = cv2.resize(img_array, (SIZE, SIZE))
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        class_num = int(labels['label'][labels[labels['name']==img].index.values])
        index_in_table = labels.index[labels['name'] == img] + 2
#         rotate = random.randint(-45, 45)
#         img_array = img_array.rotate(rotate, expand=False) 
        X.append(img_array)
        y.append(class_num)
        files_array.append(img)
#         plt.imshow(img_array)
#         plt.show()
#         break
    print(len(X))
    print(len(y))
    
creating_training_dataset(X,y,DATADIR, labels, SIZE)

In [ ]:
X = np.array(X).reshape(-1, SIZE, SIZE, 3)
y = np.array(y)

X, X_test, y, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
model_ResNet50 = keras.applications.ResNet50(input_shape=(SIZE, SIZE, 3), weights='ImageNet', include_top=True)


x = model_ResNet50.output
x = keras.layers.Dense(2, activation='softmax')(x)

model_ResNet50 = keras.Model(model_ResNet50.input, x)
model_ResNet50.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model_ResNet50.summary()

In [ ]:
updatedModel.fit(X,y, epochs=10)

In [ ]:
PREDICTDATADIR = '/content/data256/test'
X_predict = []
predict_names = []
path = os.path.join(PREDICTDATADIR)
for img in os.listdir(PREDICTDATADIR):
    img_array = cv2.imread(os.path.join(path,img))
    img_array = cv2.resize(img_array, (SIZE, SIZE))
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    X_predict.append(img_array)
    predict_names.append(img)
#     plt.imshow(img_array)
#     plt.show()
#     break
print(len(X_predict))
print(len(predict_names))

In [ ]:
X_predict = np.array(X_predict).reshape(-1, SIZE, SIZE, 3)
predictions = updatedModel.predict(X_predict)
new_predictions = []
for i in range(0, len(predictions)):
    new_predictions.append(np.argmax(predictions[i]))
print(len(predict_names))
print(len(predictions))
submit_df = pd.DataFrame([[name, pred] for name, pred in zip(predict_names, new_predictions)], columns=['name', 'pred'])
print(len(submit_df))

submit_df = submit_df.sort_values(['name'])
submit_df.to_csv("submit.csv", index=False)
submit_df.sample(10)